# Building, backtesting and deploying Autonomous Trader using mt5se
### For an introduction to mt5se, see: https://paulo-al-castro.github.io/mt5se/

In [1]:
import mt5se as se
import pandas as pd
connected=se.connect()
if connected:
    print('Ok!! It is connected to se exchange!!')
else:
    print('Something went wrong! It is NOT connected to se!!')



Ok!! It is connected to se exchange!!


## Let's create a Simple Algorithmic Trader 

In [2]:
import mt5se as se

class MultiAssetTrader(se.Trader):
    def trade(self,dbars):
        assets=dbars.keys()
        orders=[]
        for asset in assets:
            bars=dbars[asset]
            curr_shares=se.get_shares(asset)
            money=se.get_balance()/len(assets) # divide o saldo em dinheiro igualmente entre os ativos
            # number of shares that you can buy of asset 
            price=se.get_last(bars)
            free_shares=se.get_affor_shares(asset,price,money)
            rsi=se.tech.rsi(bars)
            if rsi>=70 and free_shares>0: 
                order=se.buyOrder(asset,free_shares)
            elif  rsi<70 and curr_shares>0:
                order=se.sellOrder(asset,curr_shares)
            else:
                order=None
            if order!=None:
                orders.append(order)
        return orders  


trader=MultiAssetTrader() # also available in se.sampleTraders.MultiAssetTrader()
print(trader)
 

## Setup and check a backtest!

In [3]:
# sets Backtest options 
prestart=se.date(2018,12,10)
start=se.date(2019,1,10)
end=se.date(2019,2,27)
capital=1000000
results_file='data_equity_file.csv'
verbose=False 
assets=['GOOG','AAPL']
# Use True if you want debug information for your Trader 
#sets the backtest setup
period=se.DAILY 
 # it may be se.INTRADAY (one minute interval)
bts=se.backtest.set(assets,prestart,start,end,period,capital,results_file,verbose)
if se.backtest.checkBTS(bts): # check if the backtest setup is ok!
    print('Backtest Setup is Ok!')
else:
    print('Backtest Setup is NOT Ok!')



Backtest Setup is Ok!


## Run the Backtest

In [4]:

# Running the backtest
df= se.backtest.run(trader,bts)   
# run calls the Trader. setup and trade (once for each bar)

#print the results
print(df)
print(bts['capital'])


'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG', 'AAPL'], 'capital': 1000000.0, 'shares_GOOG': 0.0, 'shares_AAPL': 0.0, 'curr': 4}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG', 'AAPL'], 'capital': 1000000.0, 'shares_GOOG': 0.0, 'shares_AAPL': 0.0, 'curr': 5}
Esta em backtest. bts= {'verbose': False, 'prestart': datetime.datetime(2018, 12, 10, 0, 0), 'start': datetime.datetime(2019, 1, 10, 0, 0), 'end': datetime.datetime(2019, 2, 27, 0, 0), 'type': 1, 'file': 'data_equity_file.csv', 'assets': ['GOOG', 'AAPL'], 'capital': 1000000.0, 'shares_GOOG': 0.0, 'shares_AAPL': 0.0, 'curr': 5}
Esta em backtest. bts= {'verbose': False, 'prestart': d

## Evaluate the Backtest result

In [5]:
# evaluates the backtest results
se.backtest.evaluate(df)



 -----------------------   Backtest Report  ------------------------------- 

Total Return (%)=2.07 in 31 bars 
Average Bar Return (%)=0.07  
Std Deviation of returns (%) =0.8539

 ----------------------        End of Report     -------------------------------- 



## Deploying a Trader for operation on real (or simulated ) Accounts 

In [2]:
import mt5se as se

class MultiAssetTrader(se.Trader):  # It is the same trader!!! There is no need to change it from backtest to live operation!!!!
    def trade(self,dbars):
        assets=dbars.keys()
        orders=[]
        for asset in assets:
            bars=dbars[asset]
            curr_shares=se.get_shares(asset)
            money=se.get_balance()/len(assets) # divide o saldo em dinheiro igualmente entre os ativos
            # number of shares that you can buy of asset 
            price=se.get_last(bars)
            free_shares=se.get_affor_shares(asset,price,money)
            rsi=se.tech.rsi(bars)
            if rsi>=70 and free_shares>0: 
                order=se.buyOrder(asset,free_shares)
            elif  rsi<70 and curr_shares>0:
                order=se.sellOrder(asset,curr_shares)
            else:
                order=None
            if order!=None:
                orders.append(order)
        return orders  


trader=MultiAssetTrader() # also available in se.sampleTraders.MultiAssetTrader()
print(trader)
 

# Setup the operation!
### Observe that there are some differences between setuping a backtest and a live operation, see below!

In [5]:

    #account data
login=None #'your account Number!' 
password=None # 'Guess what is it..'
# if login and password are not defined (i.e. None), it will use the default account defined in Metatrader
#trading data
assets=['GOOG','AAPL','TSLA']
endTime=se.now(minOffset=1)   # it will run for one minute!
#endTime=se.now(hourOffset=1,minOffset=30) # the trader will run for 1:30h after started!
 
capital=100000
data_file='data_equity_file.csv'
verbose=False
timeFrame=se.INTRADAY
delay=20 # seconds to wait between trade calls
waitForOpen=True
mem=10 
# number of bars to take into account


# define operation setup (ops)  (no se.backtest.set)
ops=se.operations.set(assets,capital,endTime,mem,timeFrame,data_file,verbose,delay,waitForOpen)
print(ops)


{'waitForOpen': True, 'verbose': False, 'delay': 15, 'mem': 10, 'end': datetime.datetime(2021, 4, 29, 14, 28, 36, 585299), 'type': 2, 'file': 'data_equity_file.csv', 'assets': ['GOOG', 'AAPL', 'TSLA'], 'capital': 100000.0}


## Connect to the Stock Exchange thorugh your Broker and execute trader through given Account


In [6]:
#Connect to the Stock Exchange
#se.connect()

#executes the trader, according setup
se.operations.run(trader,ops)

TypeError: '>' not supported between instances of 'NoneType' and 'int'

## Next,  see our notebook about building AI based Autonomous Trader